# FERNANDO LEON FRANCO

In [34]:
import numpy as np
import polars as pl
from bs4 import BeautifulSoup
import os
from colorstreak import Logger
import re
from dataclasses import dataclass


# MEJORAR LA CARGA DESDE UN CSV

In [35]:
def generar_csv(path_lectura, path_guardado):
    # Obtener la ruta absoluta del directorio actual
    # Leerlo y convertirlo en un DataFrame .csv para manipularlo mejor
    with open(path_lectura, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        
    data = [line.strip().split(":::") for line in lines]


    df = pl.DataFrame(data, schema=["id", "genero", "pais"], orient="row")


    # Guardar el DataFrame como un archivo .csv
    df.write_csv(path_guardado)
    return True

# ==================== Configuración de rutas =====================


base_path = os.getcwd()

# ===================== Creación de CSV de datos de prueba =====================
ruta_lectura_prueba = os.path.join(base_path, "data/author_profiling/es_test/truth.txt")
ruta_guardado_prueba = os.path.join(base_path, "data/author_profiling/es_test/truth_test.csv")

creado = generar_csv(ruta_lectura_prueba, ruta_guardado_prueba)

if creado:
    Logger.info(f'Archivo CSV creado en: "{ruta_guardado_prueba}"')
# ==============================================================================


# ===================== Creación de CSV de datos de entrenamiento ==============
ruta_lectura_entrenamiento = os.path.join(base_path, "data/author_profiling/es_train/truth.txt")
ruta_guardado_entrenamiento = os.path.join(base_path, "data/author_profiling/es_train/truth_train.csv")

creado_entrenamiento = generar_csv(ruta_lectura_entrenamiento, ruta_guardado_entrenamiento)
if creado_entrenamiento:
    Logger.info(f'Archivo CSV creado en: "{ruta_guardado_entrenamiento}"')
# ==============================================================================



[INFO] Archivo CSV creado en: "/Users/ferleon/Github/semestre_v/procesamiento_lenguaje/data/author_profiling/es_test/truth_test.csv"
[INFO] Archivo CSV creado en: "/Users/ferleon/Github/semestre_v/procesamiento_lenguaje/data/author_profiling/es_train/truth_train.csv"


# CARGAR LOS INDICES DESDE TRUTH DESDE EL CSV

In [ ]:
def cargar_xml(id_archivo, train=True):
    ruta_base = os.path.join(base_path, "data/author_profiling/es_test" if not train else "data/author_profiling/es_train")
    ruta_archivo = os.path.join(ruta_base, f"{id_archivo}.xml")
    Logger.info(f"ruta archivo: {ruta_archivo}")
    
    with open(ruta_archivo, 'r', encoding='utf-8') as file:
        xml_text = file.read()

    return xml_text

# ===================== Cargar CSV de datos de entrenamiento ==================
 
df_train = pl.read_csv(ruta_guardado_entrenamiento)
df_train.head()

id_archivo = df_train['id'][0]
id_archivo = "73de7fb176f5557fd54a3c5b335cbe00"
Logger.debug(f'ID del archivo: {id_archivo}')



xml_text = cargar_xml(id_archivo)
#Logger.info(f'Contenido del archivo: {xml_text[:1000]}...')  

# ===================== Procesamiento del XML =====================
diccionario_idioma = {'es': 'Español', 'en': 'Inglés', 'fr': 'Francés', 'de': 'Alemán', 'it': 'Italiano', 'nl': 'Neerlandés'}

soup = BeautifulSoup(xml_text, 'lxml-xml')   
lang = str(soup.author.get('lang'))
idioma = diccionario_idioma[lang] if lang in diccionario_idioma else lang
Logger.info(f'Idioma del autor: {idioma}')

documentos = soup.find_all('document') # Obtenemos todsa las etiquetas <document>
tweets = [doc.get_text(separator=" ", strip=True) for doc in documentos] # Extraemos el texto de cada documento


# Expresiones regulares para limpiar tweets
@dataclass
class ExpresionRegular:
    url: re.Pattern = re.compile(r'https?://\S+')
    mencion: re.Pattern = re.compile(r'@\w+')
    hashtag: re.Pattern = re.compile(r'#\w+')
    espacio: re.Pattern = re.compile(r'\s+')
    especiales: re.Pattern = re.compile(r'[^a-zA-Z0-9áéíóúÁÉÍÓÚñÑüÜ\s]')

    @classmethod
    def limpiar_tweet(cls, tweet):
        tweet = tweet.lower()
        tweet = cls.url.sub(' ', tweet)
        tweet = cls.mencion.sub(' ', tweet)
        tweet = cls.hashtag.sub(' ', tweet)
        tweet = cls.espacio.sub(' ', tweet).strip()
        return tweet

limpiados = [ExpresionRegular.limpiar_tweet(t) for t in tweets]

for i, tweet in enumerate(limpiados):
    Logger.debug(f'Tweet {i+1} | len: {len(tweet)} | {tweet} |')



[DEBUG] ID del archivo: 73de7fb176f5557fd54a3c5b335cbe00
[INFO] ruta archivo: /Users/ferleon/Github/semestre_v/procesamiento_lenguaje/data/author_profiling/es_train/73de7fb176f5557fd54a3c5b335cbe00.xml
[INFO] Idioma del autor: Español
[DEBUG] Tweet 1 | len: 101 | aún sin cerrar la boca😱 tras ver el espectáculo de en qué puesta en escena, qué realización, qué todo |
[DEBUG] Tweet 2 | len: 112 | me reitero...creando espectáculo los norteamericanos son los reyes👑 querido y sin salir de casa, desde mi sofá📺😜 |
[DEBUG] Tweet 3 | len: 40 | a ti por la sensatez y la valentía... ;) |
[DEBUG] Tweet 4 | len: 116 | de los artículos + coherentes que he leído de toda esta polémica y escrito x una madre, ya no pueden decir que si n… |
[DEBUG] Tweet 5 | len: 111 | 0:45 de la noche y llevamos más de 1 hora parados en la me gustaría saber qué está pasando... pq a pesar de la… |
[DEBUG] Tweet 6 | len: 50 | gracias a ti alba, por vernos y por tus palabras☺️ |
[DEBUG] Tweet 7 | len: 93 | y volvemos a tene